# Attention 

### I have build the image-captioning project to uderstand attnetion in detail you check that out [here](https://github.com/saurabhaloneai/image-cap.git) (also written a small blog)

- by 
